# 正式对接天擎 

In [112]:
%%time
# coding=utf8
import pymysql
import pymssql 
import numpy as np
import pandas as pd
import pickle
import redis
import datetime as dtt
# from cma.music.DataQueryClient import DataQueryClient
import json
class station_zdz:
    def __init__(self):
        self.userId = "BEHZ_TZSJ_TZSERVICE" 
        self.pwd = "Liyuan3970!@" 
        self.dataFormat = "json"
        self.citys = ["331000"]
    def decode_time_str(self,timesdelay):
        timestr = lambda x:str(x) if x >= 10 else "0"+str(x)
        today = dtt.datetime.utcnow()
        end_time = today.strftime('%Y-%m-%d %H:%M:%S') 
        offset = dtt.timedelta(minutes=-timesdelay)
        offday = (today + offset)
        offday_label = offday.strftime('%Y%m%d%H%M') +"00"
        today_label =  today.strftime('%Y%m%d%H%M') +"00"
        #return offday_label,today_label     
        timeranges = "[" + offday_label + "," + today_label + "]"
        return timeranges
    # 单站数据解析 ---------------------------------------      --------------------------------------
    def single_data(self,value,station_id,plot_range):
        # value,station_id,timesdelay 降水 站id 时间       
        timelabel = plot_range
        client = DataQueryClient()
        interfaceId = "getSurfEleByTimeRangeAndStaID"
        # 元素字典
        elem = {
            "rain":"Datetime,Station_Id_C,PRE,Station_Name,Q13011",# 降水
            "wind":"Datetime,Station_Id_C,WIN_S_Gust_Max,WIN_D_Gust_Max,Station_Name,Q11201,Q11202", #风力 
            "windave":"Station_Name,Station_Id_C,Datetime,WIN_D_Avg_1mi,WIN_S_Avg_1mi,Q11288,Q11289",
            "temp":"Datetime,Station_Id_C,TEM,Station_Name,Q12001",# 气温
            "tmax":"Datetime,Station_Id_C,TEM,Station_Name,Q12001",# 气温
            "tmin":"Datetime,Station_Id_C,TEM,Station_Name,Q12001",# 气温
            "dpt":"Station_Name,Station_Id_C,Datetime,DPT,Q12003",
            "view":"Datetime,Station_Id_C,VIS_HOR_1MI,Station_Name,Q20001_701_01",# 能见度
            "rhu":"Station_Name,Station_Id_C,Datetime,RHU,Q13003",
            "vap":"Station_Name,Station_Id_C,Datetime,VAP,Q13004",
            "Snow_Depth":"Station_Name,Station_Id_C,Datetime,Snow_Depth,Q13013",
            "prs":"Station_Name,Station_Id_C,Datetime,PRS,Q10004",
            "PRS_Sea": "Station_Name,Station_Id_C,Datetime,PRS_Sea,Q10051"        
        }
        rename = {
            "rain":"TTime,IIIII,Ri,StationName,verify",
            "temp":"TTime,IIIII,T,StationName,verify",
            "tmax":"TTime,IIIII,T,StationName,verify",
            "tmin":"TTime,IIIII,T,StationName,verify",
            "wind":"TTime,IIIII,fFy,dFy,StationName,verify1,verify2",
            "view":"TTime,IIIII,V,StationName,verify",
        }
        params = {
            'dataCode':"SURF_CHN_MUL_MIN",#SURF_CHN_MUL_HOR
            'elements':elem[value],  
            #'timeRange':"[20230118000000,20230119000000]",测试k8111的11级风
            'timeRange':timelabel,
            'staIds':station_id,
            'orderBy':"Datetime",
            'limitCnt':"50000"
        }
        result = client.callAPI_to_serializedStr(self.userId, self.pwd, interfaceId, params, self.dataFormat)
        result_json = json.loads(result)
        clomns = elem[value].split(",")
        data = pd.DataFrame(result_json['DS'],columns=clomns)
        data.columns = rename[value].split(",")
        return data        
    def single_station(self,value,station_id,plot_time):
        c_year = dtt.datetime.now().year
        c_month = dtt.datetime.now().month
        c_day = dtt.datetime.now().day
        # Disassemble the date
        date_list = plot_time.split(" ")[0].split("-")
        t_year = int(date_list[0])
        t_month = int(date_list[1])
        t_day = int(date_list[2])
        final = False
        if c_year == t_year and c_month == t_month and c_day == t_day:
            # 今天 --- 比较复杂
            offset = dtt.timedelta(minutes=-60*8)
            date_now = dtt.datetime.strptime(plot_time,"%Y-%m-%d %H:%M:%S")
            # 判断时间
            yesdelay = dtt.timedelta(days=-1)
            start = (date_now + yesdelay).strftime('%Y-%m-%d %H:%M:%S')
            end = plot_time
            today = end[0:10] + " 20:00:00"
            yesday = start[0:10] + " 20:00:00"
            hours = dtt.datetime.strptime(plot_time,'%Y-%m-%d %H:%M:%S').hour
            # 数据
            date_utc = (date_now + offset)
            delay = dtt.timedelta(minutes=-60*24)
            date_start = (date_utc+delay).strftime('%Y%m%d%H%M') +"00"
            date_end = date_utc.strftime('%Y%m%d%H%M') +"00"
            plot_range = "["+date_start +","+ date_end +"]"
            data = self.single_data(value,station_id,plot_range)
            data['TTime'] = pd.to_datetime(data['TTime'])
            data['TTime'] = data['TTime'] + dtt.timedelta(hours=8)
            if value=="wind":
                data['Year'] = data['TTime'].dt.year
                data['Month'] = data['TTime'].dt.month
                data['Day'] = data['TTime'].dt.day
                data['Hour'] = data['TTime'].dt.hour
                grouped_IIiii = data.groupby(['Day','Hour'])
                wind_time = []
                wind_dir = []
                wind_speed = []
                for i in grouped_IIiii.size().index:
                    single = grouped_IIiii.get_group(i)
                    value = single[ single['fFy']== single['fFy'].max()].head(1)
                    wind_time.append(value['TTime'].dt.strftime('%Y-%m-%d %H:%M:%S').values[0])
                    wind_speed.append(value['fFy'].values[0])
                    wind_dir.append(value['dFy'].values[0])
            else:
                wind_time = []
                wind_dir = []
                wind_speed = []  
            wind_data = pd.DataFrame(data={'TTime':wind_time,'dFy':wind_dir,'fFy':wind_speed})
            data['TTime'] = data['TTime'].dt.strftime('%Y-%m-%d %H:%M:%S')
            if hours>=20:
                # 实时数据
                now_data = data[data['TTime']>=today]
                now_data = now_data[now_data['TTime']<=end]
                if value=="wind":   
                    now_wind = wind_data[wind_data['TTime']>=today]
                    now_wind = now_wind[now_wind['TTime']<=end]
                else:
                    now_wind = now_data
                # 历史数据
                his_data = data[data['TTime']>start]
                his_data = his_data[his_data['TTime']<=today] 
                if value=="wind": 
                    his_wind = wind_data[wind_data['TTime']>start]
                    his_wind = his_wind[his_wind['TTime']<=today]
                else:
                    his_wind = his_data                    
            else:
                # 实时数据
                now_data = data[data['TTime']>=yesday]
                now_data = now_data[now_data['TTime']<=end]
                if value=="wind":
                    now_wind = wind_data[wind_data['TTime']>=yesday]
                    now_wind = now_wind[now_wind['TTime']<=end]
                else:
                    now_wind = now_data
                # 历史数据
                his_data = data[data['TTime']>start]
                his_data = his_data[his_data['TTime']<=yesday]
                if value=="wind": 
                    his_wind = wind_data[wind_data['TTime']>start]
                    his_wind = his_wind[his_wind['TTime']<=yesday] 
                else:
                    his_wind = his_data
        else:
            # 过去 进查询一个
            offset = dtt.timedelta(minutes=-60*8)
            date_now = dtt.datetime.strptime(plot_time,"%Y-%m-%d %H:%M:%S")
            date_utc = (date_now + offset)
            delay = dtt.timedelta(minutes=-60*24)
            date_start = (date_utc+delay).strftime('%Y%m%d') +"200000"
            date_end = date_utc.strftime('%Y%m%d') +"200000"
            plot_range = "["+date_start +","+ date_end +"]"
            data = self.single_data(value,station_id,plot_range)
            data['TTime'] = pd.to_datetime(data['TTime'])
            if value=="wind":
                data['Year'] = data['TTime'].dt.year
                data['Month'] = data['TTime'].dt.month
                data['Day'] = data['TTime'].dt.day
                data['Hour'] = data['TTime'].dt.hour
                grouped_IIiii = data.groupby(['Day','Hour'])
                wind_time = []
                wind_dir = []
                wind_speed = []
                for i in grouped_IIiii.size().index:
                    single = grouped_IIiii.get_group(i)
                    value = single[ single['fFy']== single['fFy'].max()].head(1)
                    wind_time.append(value['TTime'].dt.strftime('%Y-%m-%d %H:%M:%S').values[0])
                    wind_speed.append(value['fFy'].values[0])
                    wind_dir.append(value['dFy'].values[0])
            else:
                wind_time = []
                wind_dir = []
                wind_speed = [] 
            data['TTime'] = data['TTime'].dt.strftime('%Y-%m-%d %H:%M:%S')
            wind_data = pd.DataFrame(data={'TTime':wind_time,'dFy':wind_dir,'fFy':wind_speed})
            now_wind = wind_data
            his_wind = wind_data  
            now_data = data
            his_data = data           
        history = his_data.to_json(orient='values',force_ascii=False)
        nowdata = now_data.to_json(orient='values',force_ascii=False)
        windhis = his_wind.to_json(orient='values',force_ascii=False)
        windnow = now_wind.to_json(orient='values',force_ascii=False)      
        return nowdata,history,windhis,windnow
    # 单站数据解析 -----------------------     ------------------------------------------------------
    # 快报服务 -----------------------     ------------------------------------------------------
    def comput_city(self,city_code,start,end):
        """快报或者统计数据的接口"""
        # getSurfEleInRegionByTime
        date_start = dtt.datetime.strptime(start,'%Y-%m-%d %H:%M:%S') 
        date_end = dtt.datetime.strptime(end,'%Y-%m-%d %H:%M:%S') 
        offset = dtt.timedelta(minutes=-60*8)
        label_start = (date_start + offset).strftime('%Y%m%d%H%M') + "00"
        label_end = (date_end + offset).strftime('%Y%m%d%H%M') + "00"   
        labels = "[" + label_start + "," + label_end + "]"
        #timelabel = self.decode_time_str(timesdelay)
        client = DataQueryClient()
        interfaceId = "statSurfEleInRegion"  
        params = {
            'dataCode':"SURF_CHN_MUL_MIN",  #SURF_CHN_MUL_HOR
            'adminCodes':"331000",
            'elements':"Province,City,Cnty,Town,Lat,Lon,Alti,Station_levl,Station_Name,Station_Id_C,",
            'timeRange':labels,
            'statEles':'MAX_TEM,MIN_TEM,SUM_PRE,MAX_PRE_1h,MIN_VIS_HOR_1MI,MAX_WIN_S_Gust_Max,MAX_WIN_S_Avg_2mi',
            #'orderBy':"MAX_WIN_S_Inst_Max:desc",
            'staLevels':"011,012,013,014,015,016", # 12国家站 14区域站
            'limitCnt':"100000000"
        }
        result = client.callAPI_to_serializedStr(self.userId, self.pwd, interfaceId, params, self.dataFormat)
        #rint(result)
        result_json = json.loads(result)
        clomns = "Province,City,Cnty,Town,Lat,Lon,Alti,Station_levl,Station_Name,Station_Id_C,MAX_TEM,MIN_TEM,SUM_PRE,MAX_PRE_1h,MIN_VIS_HOR_1MI,MAX_WIN_S_Gust_Max,MAX_WIN_S_Avg_2mi".split(",")
        data = pd.DataFrame(result_json['DS'],columns=clomns)
        data.columns = "Province,City,Cnty,Town,Lat,Lon,Height,Stationlevl,StationName,IIIII,tmax,tmin,rain,rainhour,view,wind,windave".split(",")
        return data
    def text_rain(self,data,city_code):
        bins=[0,30,40,50,80,100,150,200,300,500,2000]
        labels=['≥0毫米','≥30毫米','≥40毫米','≥50毫米','≥80毫米','≥100毫米','≥150毫米','≥200毫米','≥300毫米','≥500毫米']
        rain = data[(data['rain']>0) & (data['rain']<5009)]
        rain['rank']=pd.cut(rain['rain'],bins,right=False,labels=labels)
        del rain['Unnamed: 0'] # 天擎不需要
        rain.reset_index(drop=True)
        if city_code in self.citys:
            cnty = rain.groupby(['Cnty'])['rain'].mean().to_dict()
        else:
            cnty = rain.groupby(['Town'])['rain'].mean().to_dict()
        rain_max = rain.sort_values(by="rain",ascending=False).head(10).to_dict()
        rain_hour = rain.sort_values(by="rainhour",ascending=False).head(5).to_dict()
        town_max = rain.groupby(['Town','Cnty'])['rain'].max().sort_values(ascending=False).head(10)
        town_max_str = []
        for i in town_max.index.to_list():
            single = (i[0] +"-" +  i[1]+ ":" +str(town_max[i]) + "毫米")
            town_max_str.append(single)
        rank_station = {
            '≥0毫米':len(rain[(rain['rain']>0) & (rain['rain']<30)].value_counts()),
            '≥30毫米':len(rain[(rain['rain']>=30) & (rain['rain']<40)].value_counts()),
            '≥40毫米':len(rain[(rain['rain']>=40) & (rain['rain']<50)].value_counts()),
            '≥50毫米':len(rain[(rain['rain']>=50) & (rain['rain']<80)].value_counts()),
            '≥80毫米':len(rain[(rain['rain']>=80) & (rain['rain']<100)].value_counts()),
            '≥100毫米':len(rain[(rain['rain']>=100) & (rain['rain']<150)].value_counts()),
            '≥150毫米':len(rain[(rain['rain']>=150) & (rain['rain']<200)].value_counts()),
            '≥200毫米':len(rain[(rain['rain']>=200) & (rain['rain']<300)].value_counts()),
            '≥300毫米':len(rain[(rain['rain']>=300) & (rain['rain']<500)].value_counts()),
            '≥500毫米':len(rain[(rain['rain']>=500) & (rain['rain']<2000)].value_counts())
        }
        # 乡镇个数统计
        town_group = rain.groupby(['Town'])['rain'].max()  
        dict_town = {'Town':town_group.index,'rain':town_group.values}
        town_range = pd.DataFrame(data =dict_town)
        town_range['rank']=pd.cut(town_range['rain'],bins,right=False,labels=labels) 
        rank_town = {
            '≥0毫米':len(town_range[(town_range['rain']>0) & (town_range['rain']<30)].value_counts()),
            '≥30毫米':len(town_range[(town_range['rain']>=30) & (town_range['rain']<40)].value_counts()),
            '≥40毫米':len(town_range[(town_range['rain']>=40) & (town_range['rain']<50)].value_counts()),
            '≥50毫米':len(town_range[(town_range['rain']>=50) & (town_range['rain']<80)].value_counts()),
            '≥80毫米':len(town_range[(town_range['rain']>=80) & (town_range['rain']<100)].value_counts()),
            '≥100毫米':len(town_range[(town_range['rain']>=100) & (town_range['rain']<150)].value_counts()),
            '≥150毫米':len(town_range[(town_range['rain']>=150) & (town_range['rain']<200)].value_counts()),
            '≥200毫米':len(town_range[(town_range['rain']>=200) & (town_range['rain']<300)].value_counts()),
            '≥300毫米':len(town_range[(town_range['rain']>=300) & (town_range['rain']<500)].value_counts()),
            '≥500毫米':len(town_range[(town_range['rain']>=500) & (town_range['rain']<2000)].value_counts())
        }
        rain_dir = {
            "average_city":cnty,
            "station_index":rain_max,
            "hour_max":rain_hour,
            "town_max":town_max_str,
            "station_count":rank_station,
            "town_count":rank_town        
        }
        return rain_dir
    def text_wind(self,data,city_code):
        bins=[0,10.7,13.8,17.1,20.7,24.4,28.4,32.6,36.9,41.4,46.1,50.9,56,80]
        labels=['6级风以下','6级风','7级风','8级风','9级风','10级风','11级风','12级风','13级风','14级风','15级风','16级风','17级风']
        wind = data[(data['wind']>0) & (data['wind']<5009)]
        wind['rank']=pd.cut(wind['wind'],bins,right=False,labels=labels)
        del wind['Unnamed: 0'] # 天擎不需要
        wind.reset_index(drop=True)# 天擎不需要
        wind_max = wind.sort_values(by="wind",ascending=False).head(10).to_dict()
        rank_station = {
            '6级风以下':len(wind[(wind['wind']>0) & (wind['wind']<10.7)].to_dict()),
            '6级风':len(wind[(wind['wind']>=10.7) & (wind['wind']<13.8)].value_counts()),
            '7级风':len(wind[(wind['wind']>=13.8) & (wind['wind']<17.1)].value_counts()),
            '8级风':len(wind[(wind['wind']>=17.1) & (wind['wind']<20.7)].value_counts()),
            '9级风':len(wind[(wind['wind']>=20.7) & (wind['wind']<24.4)].value_counts()),
            '10级风':len(wind[(wind['wind']>=24.4) & (wind['wind']<28.4)].value_counts()),
            '11级风':len(wind[(wind['wind']>=28.4) & (wind['wind']<32.6)].value_counts()),
            '12级风':len(wind[(wind['wind']>=32.6) & (wind['wind']<36.9)].value_counts()),
            '13级风':len(wind[(wind['wind']>=36.9) & (wind['wind']<41.4)].value_counts()),
            '14级风':len(wind[(wind['wind']>=41.4) & (wind['wind']<46.1)].value_counts()),
            '15级风':len(wind[(wind['wind']>=46.1) & (wind['wind']<50.9)].value_counts()),
            '16级风':len(wind[(wind['wind']>=50.9) & (wind['wind']<56.4)].value_counts()),
            '17级及以上风':len(wind[(wind['wind']>=56.4) & (wind['wind']<80)].value_counts())
        }
        return rank_station
    def text_tmax(self,data,city_code):
        bins=[-40,35,37,40,80]
        labels=['正常','35度以上','37度以上','40度以上']
        tmax = data[(data['tmax']>-40) & (data['tmax']<5009)]
        tmax['rank']=pd.cut(tmax['tmax'],bins,right=False,labels=labels)
        del tmax['Unnamed: 0'] # 天擎不需要
        tmax.reset_index(drop=True)# 天擎不需要
        tmax_max = tmax.sort_values(by="tmax",ascending=False).head(10).to_dict()
        rank_tmax = {
           '正常':len(tmax[(tmax['tmax']>-40) & (tmax['tmax']<35)].to_dict()),
            '35度以上':len(tmax[(tmax['tmax']>=35) & (tmax['tmax']<37)].value_counts()),
            '37度以上':len(tmax[(tmax['tmax']>=37) & (tmax['tmax']<40)].value_counts()),
            '40度以上':len(tmax[(tmax['tmax']>=40) & (tmax['tmax']<80)].value_counts()) 
        }
        return rank_tmax
    def text_tmin(self,data,city_code):
        bins=[-20,-5,0,2,4]
        labels=['正常','35度以上','37度以上','40度以上']
        tmin = data[(data['tmin']>-40) & (data['tmin']<5009)]
        tmin['rank']=pd.cut(tmin['tmin'],bins,right=False,labels=labels)
        del tmin['Unnamed: 0'] # 天擎不需要
        tmin.reset_index(drop=True)# 天擎不需要
        tmin_max = tmin.sort_values(by="tmin",ascending=False).head(10).to_dict()
        rank_tmin = {
           '正常':len(tmin[(tmin['tmin']>-40) & (tmin['tmin']<35)].to_dict()),
            '35度以上':len(tmin[(tmin['tmin']>=35) & (tmin['tmin']<37)].value_counts()),
            '37度以上':len(tmin[(tmin['tmin']>=37) & (tmin['tmin']<40)].value_counts()),
            '40度以上':len(tmin[(tmin['tmin']>=40) & (tmin['tmin']<80)].value_counts()) 
        }
        return rank_tmin
    def text_view(self,data,city_code):
        bins=[0,50,200,500,99999]
        labels=['强浓雾','浓雾','大雾','正常']
        view = data[(data['view']>-40) & (data['view']<5009)]
        view['rank']=pd.cut(view['view'],bins,right=False,labels=labels)
        del view['Unnamed: 0'] # 天擎不需要
        view.reset_index(drop=True)# 天擎不需要
        view_max = view.sort_values(by="view",ascending=False).head(10).to_dict()
        rank_view = {
           '强浓雾':len(view[(view['view']>0) & (view['view']<50)].to_dict()),
            '浓雾':len(view[(view['view']>=50) & (view['view']<200)].value_counts()),
            '大雾':len(view[(view['view']>=200) & (view['view']<500)].value_counts()),
            '正常':len(view[(view['view']>=500) & (view['view']<99990)].value_counts()) 
        }
        return rank_view
    def text_msg(self,city_code):
        data = pd.read_csv("service.csv")
        output = self.text_rain(data,city_code)
        output = self.text_wind(data,city_code)
        output = self.text_tmax(data,city_code)
        output = self.text_tmin(data,city_code)
        output = self.text_view(data,city_code)
        return output 
    def upload_mysql_data(self,start,end):
        client = DataQueryClient()
        interfaceId = "getSurfEleInRectByTimeRange"
        params = {
            'dataCode':"SURF_CHN_MUL_MIN",  #SURF_CHN_MUL_HOR
            'elements':"Cnty,Province,Town,Station_levl,Station_Name,City,Station_Id_C,Lat,Lon,Alti,Datetime,PRS,PRS_Sea,WIN_S_Gust_Max,WIN_D_Gust_Max,WIN_D_Avg_2mi,WIN_S_Avg_2mi,TEM,DPT,RHU,VAP,PRE,Snow_Depth,VIS_HOR_1MI,CLO_Cov,CLO_Height_LoM",
            'timeRange':'[20230701000000,20230701000300]',
            'minLat':"25",
            'minLon':"115",
            'maxLat':"35",
            'maxLon':"125",
            'limitCnt':"100000000"
        }
        result = client.callAPI_to_serializedStr(self.userId, self.pwd, interfaceId, params, self.dataFormat)
        result_json = json.loads(result)
        clomns = params['elements'].split(",")
        data = pd.DataFrame(result_json['DS'],columns=clomns)
        return data

    
worker = station_zdz()
# 单站查询
# plot_time = '2023-06-30 11:01:00'   
# value = "wind"        
# station_id = "K8111"
# nowdata,history,windhis,windnow = worker.single_station(value,station_id,plot_time)
# 单站查询

# 气象快报
city_code = "331000"
start = '2023-07-01 20:52:00'
end = '2023-07-02 20:52:00'
# data = worker.comput_city(city_code,start,end) # 统计数据

worker.text_msg(city_code) # 返回文字稿
#data
# 气象快报

# 获取区域数据
# start = '2023-07-01 20:52:00'
# end = '2023-07-02 20:52:00'
# data = worker.upload_mysql_data(start,end)
# 获取区域数据



CPU times: user 167 ms, sys: 0 ns, total: 167 ms
Wall time: 169 ms


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:234: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:290: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:314: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

{'强浓雾': 18, '浓雾': 0, '大雾': 6, '正常': 23}

In [86]:
data

['街头镇-天台县:56.7毫米',
 '龙溪乡-天台县:46.0毫米',
 '平桥镇-天台县:29.8毫米',
 '白塔镇-仙居县:21.9毫米',
 '步路乡-仙居县:19.6毫米',
 '三合镇-天台县:18.6毫米',
 '古城街道-临海市:17.3毫米',
 '雷峰乡-天台县:16.1毫米',
 '南屏乡-天台县:15.9毫米',
 '三州乡-天台县:14.9毫米']

In [36]:
data

,Province,City,Cnty,Town,Lat,Lon,Height,Stationlevl,StationName,IIIII,tmax,tmin,rain,rainhour,view,wind,windave
0,浙江省,台州市,三门县,健跳镇,29.105,121.5181,45,14,海游港,K8845,35.8,26,0,0,999999,8.1,6.1
1,浙江省,台州市,温岭市,坞根镇,28.3028,121.2569,35,14,下呈村,K8475,35.3,27.7,0,0,999999,999999,999999
2,浙江省,台州市,温岭市,新河镇,28.4511,121.4406,6,14,长屿小学,K8465,34.6,28,0,0,999999,9.9,5.7
3,浙江省,台州市,温岭市,石塘镇,28.2717,121.7308,35,14,洛屿岛,K8488,28.1,25.6,0,0,7469,14,12.7
4,浙江省,台州市,玉环市,海山乡,28.21,121.1575,2,14,海山,K8504,31.8,27.5,0,0,1560,15.6,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,浙江省,台州市,临海市,小芝镇,28.8933,121.3694,137,14,红树林,K8644,36,24.5,0.1,0.1,999999,6.6,3.8
596,浙江省,台州市,临海市,白水洋镇,29,120.9667,804,14,安基山,K8643,30.2,23,0.1,0.1,999999,9.6,7.1
597,浙江省,台州市,温岭市,大溪镇,28.4822,121.3236,9,14,火车站,K8422,36.3,28.1,0,0,999999,9,4.6
598,浙江省,台州市,临海市,小芝镇,28.8522,121.4519,82,16,小芝,K8609,36.7,26.2,0.2,0.2,999999,6.9,4.6


# 部分对接气象快报的脚本

In [32]:
import pandas as pd

data = pd.read_csv("service.csv")

data

,Unnamed: 0,Province,City,Cnty,Town,Lat,Lon,Height,Stationlevl,StationName,IIIII,tmax,tmin,rain,view,wind,windave
0,0,浙江省,台州市,路桥区,新桥镇,28.5019,121.4036,4.0,14,新桥,K8308,34.2,27.5,0.0,999999,9.5,6.5
1,1,浙江省,台州市,三门县,浦坝港镇,28.9647,121.7919,12.0,14,三门岛,K8860,29.3,26.7,0.0,15008,18.2,14.9
2,2,浙江省,台州市,临海市,小芝镇,28.8794,121.3731,59.0,14,外蔡,K8613,35.6,24.5,1.4,999999,0.0,0.0
3,3,浙江省,台州市,三门县,亭旁镇,29.0244,121.3881,415.0,14,枧头村,K8828,36.0,24.8,0.0,999999,8.3,6.3
4,4,浙江省,台州市,天台县,白鹤镇,29.2036,120.9628,90.0,14,岭下洋,K8741,36.5,25.3,0.0,999999,8.1,4.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,595,浙江省,台州市,温岭市,城南镇,28.3278,121.4594,10.0,14,温岭南,K8447,31.0,27.4,0.0,5219,12.2,8.3
596,596,浙江省,台州市,黄岩区,头陀镇,28.6425,121.1564,60.0,14,柑橘所,K8233,999999.0,999999.0,999999.0,999999,999999.0,999999.0
597,597,浙江省,台州市,温岭市,新河镇,28.4467,121.4456,26.0,14,长屿村,K8474,35.5,27.4,0.0,999999,999999.0,999999.0
598,598,浙江省,台州市,临海市,大田街道,28.9617,121.1519,212.0,14,石门村,K8685,35.4,25.2,0.1,999999,999999.0,999999.0


In [38]:
import pandas as pd
data.to_csv("service.csv")

In [233]:
data[data['IIIII']=='K8062']

,Province,City,Cnty,Town,Lat,Lon,Height,Stationlevl,StationName,IIIII,tmax,tmin,rain,view,wind,windave


In [171]:
data

,StationName,IIIII,TTime,Ri,verify
0,二桥,K8111,2023-07-01 19:01:00,0,999999
1,二桥,K8111,2023-07-01 19:02:00,0,999999
2,二桥,K8111,2023-07-01 19:03:00,0,999999
3,二桥,K8111,2023-07-01 19:04:00,0,999999
4,二桥,K8111,2023-07-01 19:05:00,0,999999
...,...,...,...,...,...
1436,二桥,K8111,2023-07-02 18:57:00,0,999999
1437,二桥,K8111,2023-07-02 18:58:00,0,999999
1438,二桥,K8111,2023-07-02 18:59:00,0,999999
1439,二桥,K8111,2023-07-02 19:00:00,0,999999


In [137]:
data[data['Station_Id_C']=="K8111"]

,Province,City,Lat,Lon,Alti,Town,Station_levl,Station_Name,Station_Id_C,MAX_TEM,MIN_TEM,SUM_PRE,MIN_VIS_HOR_1MI,MAX_WIN_S_Gust_Max,MAX_WIN_S_Avg_2mi
370,浙江省,台州市,28.6944,121.4744,30,前所街道,14,二桥,K8111,34.7,24.1,11.7,906,31.3,18.2


In [109]:
drop = data.drop_duplicates(subset=['Station_Id_C'],keep="first")

In [110]:
drop

,Province,Town,Station_levl,Station_Name,City,Station_Id_C,Alti,WIN_D_INST_Max,MAX_TEM_Max,MIN_TEM_Min,SUM_PRE,MIN_VIS_HOR_1MI,MAX_WIN_S_Inst_Max
0,浙江省,安洲街道,14,路口,台州市,K8975,576,167,26.4,26,0,999999,0
1,浙江省,湫山乡,14,湫山,台州市,K8926,134,216,24.5,24.1,0,999999,0
2,浙江省,江南街道,14,江南,台州市,K8615,11,48,26.5,26.5,0,999999,0
4,浙江省,小芝镇,13,小芝,台州市,K8609,82,297,25.5,25.5,0,999999,0
5,浙江省,蛇蟠乡,14,蛇蟠旅游,台州市,K8830,5,0,27.3,25.9,0,5320,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
38298,浙江省,石梁镇,14,华顶,台州市,K8737,950,999999,999999,999999,999999,999999,999999
38299,浙江省,新河镇,14,屿头村,台州市,K8473,25,999998,35.4,25.3,6.8,999999,999999
38300,浙江省,白水洋镇,14,丁公园村,台州市,K8679,467,999998,31.3,23.3,0,999999,999999
38301,浙江省,雷峰乡,14,雷峰漂流,台州市,K8792,209,999998,30.9,23.6,0.5,999999,999999


In [111]:
drop[drop['Station_Id_C']=="K8111"]

,Province,Town,Station_levl,Station_Name,City,Station_Id_C,Alti,WIN_D_INST_Max,MAX_TEM_Max,MIN_TEM_Min,SUM_PRE,MIN_VIS_HOR_1MI,MAX_WIN_S_Inst_Max
48,浙江省,前所街道,14,二桥,台州市,K8111,30,0,25.7,25.4,0,5589,0


In [34]:
drop = data.drop_duplicates(subset=['Station_Id_C'],keep="first")

In [40]:
drop

,Cnty,Province,Town,Station_levl,Station_Name,City,Station_Id_C,Lat,Lon,Alti,Station_Id_d,Station_Type,PRE,TEM_Max,TEM_Min,WIN_D_INST_Max,WIN_S_Inst_Max,VIS_Min
0,椒江区,浙江省,三甲街道,14,烟草中心,台州市,K8122,28.59,121.5361,3,999999,999999,0,28.3,27,260,3.6,999998
1,黄岩区,浙江省,上郑乡,14,大寺基,台州市,K8222,28.5842,120.7997,1143,999999,999999,0,21.7,20.9,278,8.1,999998
2,黄岩区,浙江省,富山乡,14,外金,台州市,K8271,28.5078,120.8914,582,999999,999999,0,24.2,23,128,3.2,999998
3,黄岩区,浙江省,上垟乡,14,前思岙,台州市,K8274,28.4861,121.0247,185,999999,999999,0,25.2,23.7,345,0.8,999998
4,黄岩区,浙江省,院桥镇,14,院桥,台州市,K8213,28.5497,121.2981,7,999999,999999,0,26.9,26.1,347,2.6,999998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2815,仙居县,浙江省,溪港乡,14,大园村,台州市,K8992,28.4783,120.4031,456,999999,999999,0,22.6,22.4,160,3.7,999998
3183,玉环市,浙江省,干江镇,14,栈台渔港,台州市,K8566,28.1408,121.3567,48,999999,999999,0,25.9,25.7,222,7.4,999998
3273,天台县,浙江省,坦头镇,14,苍山顶,台州市,K8727,29.1514,121.1528,918,999999,999999,0,25,24.5,247,6.9,999998
4028,仙居县,浙江省,福应街道,14,青尖山,台州市,K8945,28.9164,120.7275,824,999999,999999,0,23.7,22.8,280,4.9,999998


In [3]:
from pycwr.io import read_auto

<!-- # 数据库对接天擎 -->